# Exploratory Data Analysis 

## Highlight of Trends

1. **Seasonality of Complaints**

    - Certain types of complaints display a strong seasonality trend. For example, Heat/Hot Water complaints spike in winter; Noise complaints tend to spike in summer.

        ![Complaint Volume Trend](images/seasonality_trend.png)



2. **Complaints Volume by Borough**

    - While Brooklyn has the highest total complaint volume among all boroughs, Bronx has the highest complaint volume per capita.

        ![Complaint Volume Trend](images/complaint_vol_by_borough.png)

        ![Complaint Volume Trend](images/complaint_vol_per_capita.png)



3. **Complaints Volume by Time**

    - Complaint volumes peak around midday (10 AM–2 PM) on weekdays, while weekend complaints are more evenly spread throughout the day.

        ![Complaint Volume Trend](images/complaint_timing.png)


4. **Resolution Time by Borough**

    - For complaint types that require longer resolution times (over 1.5 days), Manhattan has the slowest average turnaround.
       
        ![Complaint Volume Trend](images/resolution_by_borough.png)


5. **Resolution Time by Open Channel**

    - Resolution time has decreased over time. Complaints filed through the mobile channel have the fastest resolution time.
       
        ![Complaint Volume Trend](images/resolution_by_channel.png)


## Code

In [2]:
import pandas as pd
import plotly.express as px

In [7]:
df = pd.read_parquet("data/cache_clean_data/cleaned_data.parquet")

In [8]:
nyc_pop = pd.read_csv("data/cache_clean_data/nyc_population.csv")

### basic information

In [9]:
df.describe(include='all')

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,incident_zip,city,facility_type,...,resolution_action_updated_date,borough,x_coordinate_state_plane,y_coordinate_state_plane,open_data_channel_type,latitude,longitude,days_to_close,created_month,closed_month
count,17899464,17899464,17573983,17899464,17899464,17899464,17740931,17623448,16957069,1249011,...,17753802,17861029,1.756633e+07,1.756759e+07,17899464,1.756619e+07,1.756619e+07,1.757398e+07,17899464,17573983
unique,17899464,NaN,NaN,18,19,269,1281,609,597,2,...,NaN,6,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN
top,57461812,NaN,NaN,NYPD,New York City Police Department,Illegal Parking,Loud Music/Party,10466,BROOKLYN,N/A,...,NaN,BROOKLYN,NaN,NaN,ONLINE,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,7804209,7804209,2212267,2441866,472536,5135539,1057512,...,NaN,5354904,NaN,NaN,7206221,NaN,NaN,NaN,NaN,NaN
mean,NaN,2022-11-12 06:57:15.743928576,2022-11-27 13:35:05.623907584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022-12-02 09:19:43.148033280,NaN,1.005877e+06,2.078089e+05,NaN,4.073701e+01,-7.392193e+01,2.037774e+01,2022-10-28 02:00:11.309578240,2022-11-12 10:07:15.299705088
min,NaN,2020-01-01 00:00:00,1899-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2019-07-18 00:00:00,NaN,9.133530e+05,-2.280004e+06,NaN,2.040075e-06,-7.425495e+01,-4.460200e+04,2020-01-01 00:00:00,1899-12-01 00:00:00
25%,NaN,2021-07-05 08:07:52,2021-07-17 01:18:44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2021-07-22 00:00:00,NaN,9.938190e+05,1.844640e+05,NaN,4.067293e+01,-7.396548e+01,0.000000e+00,2021-07-01 00:00:00,2021-07-01 00:00:00
50%,NaN,2022-11-15 17:13:05,2022-12-09 13:00:53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2022-12-17 02:52:36,NaN,1.005174e+06,2.049760e+05,NaN,4.072923e+01,-7.392447e+01,0.000000e+00,2022-11-01 00:00:00,2022-12-01 00:00:00
75%,NaN,2024-04-09 07:16:59.249999872,2024-05-02 21:51:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2024-05-08 15:53:43.249999872,NaN,1.019332e+06,2.372730e+05,NaN,4.081789e+01,-7.387336e+01,3.000000e+00,2024-04-01 00:00:00,2024-05-01 00:00:00
max,NaN,2025-07-18 02:08:31,2033-03-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2025-09-19 00:00:00,NaN,3.113073e+07,2.720890e+05,NaN,4.091346e+01,-1.149235e-07,3.929000e+03,2025-07-01 00:00:00,2033-03-01 00:00:00


In [ ]:
# around 2% of complaints do not have a closed date yet. They might be newer complaints still being processed.
# around 1.5% of complaints do not have incident_zip on record.
# key columns like created_date, agency_name, complaint_type, open_data_channel_type and status have no missing values.
# most of the complaints do not have a due date.
df.isnull().sum()

unique_key                               0
created_date                             0
closed_date                         325481
agency                                   0
agency_name                              0
complaint_type                           0
descriptor                          158533
incident_zip                        276016
city                                942395
facility_type                     16650453
status                                   0
due_date                          17898150
resolution_action_updated_date      145662
borough                              38435
x_coordinate_state_plane            333137
y_coordinate_state_plane            331877
open_data_channel_type                   0
latitude                            333275
longitude                           333275
days_to_close                       325481
created_month                            0
closed_month                        325481
dtype: int64

In [6]:
print("There are", df.duplicated().sum(), "duplicated rows.")
df.drop_duplicates(inplace=True)

There are 0 duplicated rows.


In [7]:
print("There are", df[(df['closed_date']<= df['created_date']) & (df['closed_date'].notnull())].shape[0], 
      "complaints that have a closed date earlier than or equal to the created date.")

df = df[(df['closed_date'] > df['created_date']) | (df['closed_date'].isnull())]

There are 480100 complaints that have a closed date earlier than or equal to the created date.


In [8]:
cols_to_investigate = ['borough', 'agency_name','status', 'open_data_channel_type']
for col in cols_to_investigate:
    print(f"Unique values in {col}:")
    print(df[col].unique())
    print()

Unique values in borough:
['BRONX' 'QUEENS' 'BROOKLYN' 'MANHATTAN' 'STATEN ISLAND' 'Unspecified'
 None]

Unique values in agency_name:
['Department of Housing Preservation and Development'
 'New York City Police Department' 'Department of Buildings'
 'Department of Environmental Protection'
 'Department of Health and Mental Hygiene'
 'Department of Consumer and Worker Protection' 'Department of Sanitation'
 'Department of Transportation' 'Taxi and Limousine Commission'
 'Department of Parks and Recreation' 'Department of Homeless Services'
 'Economic Development Corporation' 'Department of Education'
 'Office of Technology and Innovation' 'Department for the Aging'
 'Department of Information Technology and Telecommunications'
 "Mayor's Office of Special Enforcement"
 'Operations Unit - Department of Homeless Services' '3-1-1']

Unique values in status:
['Closed' 'Assigned' 'Open' 'Pending' 'In Progress' 'Started'
 'Unspecified' 'Cancel']

Unique values in open_data_channel_type:
['ONL

In [ ]:
# check the top 10 incident_zip by complaint volume
df['incident_zip'] = df['incident_zip'].astype(str).str.zfill(5)
display(df['incident_zip'].value_counts().head(10))

incident_zip
10466    468470
11226    269453
10467    246432
10457    241643
10468    241083
11385    240000
0None    234462
10452    230220
10458    216689
11207    215907
Name: count, dtype: int64

### complaints volume by type

In [ ]:
# Certain complaint types display seasonal patterns 

# find the top 10 complaint types
top_complaint_types = df['complaint_type'].value_counts().head(10).index.tolist()

condition = (
    df['complaint_type'].isin(top_complaint_types) &
    (df['created_date'] < '2025-07-01') 
)

complaint_volume_by_month = df[condition].groupby(['created_month', 'complaint_type']).size().reset_index(name='complaint_count')

fig = px.line(
    complaint_volume_by_month, 
    x='created_month', 
    y='complaint_count', 
    color='complaint_type', 
    title='Monthly Complaint Volume by Type (Top 10 Complaint Types)'
)
fig.update_layout(width=1200, height=500)

### complaints volume by borough

In [ ]:
# Brooklyn has the highest complaint volume 

condition = (
    (df['created_date'] < '2025-07-01') &
    (df['borough'].notnull()) &
    (df['borough'].isin(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']))
)

complaint_volume_by_borough = df[condition].groupby(['created_month', 'borough']).agg({'unique_key': 'count'}).reset_index()

complaint_volume_by_borough.rename(columns={'unique_key': 'complaint_count'}, inplace=True)

fig = px.line(
    complaint_volume_by_borough, 
    x='created_month', 
    y='complaint_count', 
    color='borough', 
    title='Monthly Complaint Volume by Borough'
)
fig.update_layout(width=1200, height=500)

In [23]:
# Bronx has the highest complaint volume per 1000 residents

# Create a mapping of zip codes to boroughs
# If a zip code has multiple boroughs, keep the one with the highest complaint count
zip_borough_map = (
    df[df['borough'].notna() & (df['borough'] != 'Unspecified')]
    .groupby(['incident_zip', 'borough'])
    .size()
    .reset_index(name='complaint_count')
    .sort_values(['incident_zip', 'complaint_count'], ascending=[True, False])
    .drop_duplicates(subset=['incident_zip'], keep='first')
    .reset_index(drop=True)
)[['incident_zip', 'borough']]
 
# Use the zip_borough_map to merge with the nyc_population data to get borough populations
nyc_pop['zip'] = nyc_pop['zip'].astype(str).str.zfill(5)

merged_pop = nyc_pop.merge(zip_borough_map, left_on='zip', right_on = 'incident_zip', how='inner', suffixes=('', '_borough'))

borough_pop = merged_pop.groupby('borough')['population'].sum().reset_index()
 
# Find the complaint volume per 1000 residents for each borough
complaint_volume_by_borough = complaint_volume_by_borough.merge(borough_pop, on='borough', how='left')

complaint_volume_by_borough['complaint_per_capita_1000'] = complaint_volume_by_borough['complaint_count'] / complaint_volume_by_borough['population'] * 1000

fig = px.line(
    complaint_volume_by_borough, 
    x='created_month', 
    y='complaint_per_capita_1000', 
    color='borough', 
    title='Monthly Complaint Volume per 1000 Residents by Borough'
    )
fig.update_layout(width=1200, height=500)

In [ ]:
# In January 2025, there was a spike in the Residential noise complaints in Bronx

bronx_top_complaint_types = df[df['borough'].isin(['BRONX'])]['complaint_type'].value_counts().head(10).index.tolist()

condition = (
    (df['borough'].isin(['BRONX'])) &
    (df['complaint_type'].isin(bronx_top_complaint_types))
)

complaint_volume_bronx = df[condition].groupby(['created_month', 'complaint_type']).size().reset_index(name='complaint_count')
fig = px.line(
    complaint_volume_bronx,
    x='created_month',
    y='complaint_count',
    color='complaint_type',
    title='Complaint Volume by Type in Bronx (Top 10 Complaint Types)'
)
fig.update_layout(width=1200, height=500)


### resolution time by borough

In [ ]:
# Manhattan has the highest average resolution time 

condition = (

    (df['borough'].isin(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'])) &
    (df['status']== 'Closed')
)

resolution_time = df[condition].groupby('borough').agg({'days_to_close': 'mean'}).reset_index().sort_values(by='days_to_close', ascending=False)

fig = px.bar(
    resolution_time,
    x='borough',
    y='days_to_close',
    title='Average Resolution Time by Borough (in days)',
)
fig.update_layout(width=1200, height=500)


### resolution time by complaint type

In [ ]:
# Unsanitary conditions, street conditions, request large bulky item collection, and heat/hot water issues require longer resolution time

condition = (

    (df['borough'].isin(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'])) &
    (df['status']== 'Closed') &
    df['complaint_type'].isin(top_complaint_types)
)

resolution_time = df[condition].groupby('complaint_type').agg({'days_to_close': 'mean'}).reset_index().sort_values(by='days_to_close', ascending=False)

fig = px.bar(
    resolution_time,
    x='complaint_type',
    y='days_to_close',
    title='Average Resolution Time by Complaint Type (in days)',
)
fig.update_layout(width=1200, height=500)

In [ ]:
# Manhattan has the highest average resolution time for high-maintenance complaints 
condition = (

    (df['borough'].isin(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND'])) &
    (df['status']== 'Closed') &
    (df['complaint_type'].isin(['UNSANITARY CONDITION','Street Condition','Request Large Bulky Item Collection','HEAT/HOT WATER']))
)

resolution_time = df[condition].groupby(['borough']).agg({'days_to_close': 'mean'}).reset_index().sort_values(by='days_to_close', ascending=False)

fig = px.bar(
    resolution_time,
    x='borough',
    y='days_to_close',
    title='Average Resolution Time by Borough (in days)',
    barmode='group'  
)
fig.update_layout(width=1200, height=500)

### complaints volume by day of week and time of day

In [26]:
# 10am - 2pm during weekdays is the peak time for complaints

df['day_of_week'] = df['created_date'].dt.day_name()
df['hour_of_day'] = df['created_date'].dt.hour

heatmap = df.groupby(['day_of_week', 'hour_of_day']).size().reset_index(name='complaint_count')

days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

heatmap['day_of_week'] = pd.Categorical(heatmap['day_of_week'], categories=days_order, ordered=True)

fig = px.density_heatmap(
    heatmap,
    x='hour_of_day',
    y='day_of_week',
    z='complaint_count',
    color_continuous_scale='Viridis',
    title='Complaint Volume by Day of Week and Time of Day'
)

fig.update_layout(
    xaxis_title='Hour of Day',
    yaxis_title='Day of Week',
    width=1200,
    height=600
)

fig.update_yaxes(categoryorder='array', categoryarray=days_order)

### complaints distribution by status

In [ ]:
# Staten Island has the lowest percentage of closed complaints month-to-date
condition =  (
    (df['created_date'] >= '2025-07-01')  & 
    (df['status'].notnull()) &
    (df['borough'].isin(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN ISLAND']))
)

status_borough_counts = (
    df[condition]
    .groupby(['borough', 'status'])
    .size()
    .reset_index(name='complaint_count')
)

status_borough_counts['total_per_borough'] = status_borough_counts.groupby('borough')['complaint_count'].transform('sum')

status_borough_counts['percentage'] = status_borough_counts['complaint_count'] / status_borough_counts['total_per_borough'] * 100

fig = px.bar(
    status_borough_counts,
    x='borough',
    y='percentage',
    color='status',
    barmode='stack',
    title='Month-to-date Complaint Distribution by Status and Borough'
)

fig.update_layout(width=1200, height=500)

### complaints volume by agency

In [ ]:
# New York City Police Department handles the most complaints among all agencies
agency_counts = df['agency_name'].value_counts().reset_index()
agency_counts.columns = ['agency_name', 'complaint_count']

fig = px.bar(
    agency_counts,
    x='agency_name',
    y='complaint_count',
    title='Complaint Volume by Agency',
    text='complaint_count'
)
fig.update_layout(width=1200, height=500)

### complaints volume by open channel

In [ ]:
# Online is the most popular open channel 
condition =  (
    (df['created_date'] < '2025-07-01') &
    (df['open_data_channel_type'].notnull())
)

vol_by_channel = (
    df[condition]
    .groupby(['created_month', 'open_data_channel_type'])
    .size()
    .reset_index(name='complaint_count')
)

fig = px.bar(
    vol_by_channel,
    x='created_month',
    y='complaint_count',
    color='open_data_channel_type',
    barmode='stack',
    title='Complaint Volume by Open Channel',
)
fig.update_layout(width=1200, height=500)

### resolution time by open channel

In [20]:
# The resolution time decreases over time, with mobile channel having the lowest resolution time
condition =  (
    (df['created_date'] < '2025-07-01') &
    (df['open_data_channel_type'].isin(['MOBILE', 'ONLINE', 'PHONE'])) &
    (df['status'] == 'Closed')
)


res_time_by_channel = (
    df[condition]
    .groupby(['created_month', 'open_data_channel_type'])
    .agg({'days_to_close': 'mean'})
    .reset_index()
)

fig = px.line(
    res_time_by_channel,
    x='created_month',
    y='days_to_close',
    color='open_data_channel_type',
    title='Average Resolution Time by Open Channel',
)
fig.update_layout(width=1200, height=500)